<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/Python-Notebook-Banners/Code_challenge.png"  style="display: block; margin-left: auto; margin-right: auto;";/>
</div>

# Code challenge notebook: Loops and functions
© ExploreAI Academy

In this coding challenge, we will apply our Python programming skills to automate more complex movement of our farm equipment. We will use functions to make our code more modular, making it simpler to read, modify, and extend.

⚠️ **NOTE: This notebook should be used to work though the code challenge, but should not be submitted. Please transfer your code to `GCC_Submit_loops_and_functions.ipynb` and submit that file online. Submitting this notebook will **fail** the autograder tests.**

### Instructions

- **Do not add or remove cells in this notebook. Do not edit or remove the `### START FUNCTION` or `### END FUNCTION` comments. Do not add any code outside of the functions you are required to edit. Doing any of this will lead to a mark of 0%!**

- Answer the questions according to the specifications provided.

- Use the given cell in each question to see if your function matches the expected outputs.

- Do not hard-code answers to the questions.

- The use of StackOverflow, Google, and other online tools is permitted. However, copying a fellow student's code is not permissible and is considered a breach of the Honour code. Doing this will result in a mark of 0%.

## Introduction

Our ultimate goal is to develop fully autonomous farming equipment. These machines will not just follow preset instructions; they will make intelligent decisions based on real-time data. Imagine tractors that know when to plough, seeders that understand the best planting patterns, and irrigation systems that water crops precisely when needed.

But every grand plan starts with a first step. For us, it's creating a digital mirror of Maji Ndogo's farming ecosystem - a comprehensive **Digital Twin**. This virtual model will be our testing ground. Here, we'll experiment with different agricultural strategies, analyse their outcomes, and perfect our approaches without a single seed being sown in the real world.

To build this digital farm, we'll rely on our Python programming skills. We’ll start by digitally constructing the farm elements:

- **Fields:** We'll create grid representations of fields where each cell is a patch of land.

- **Farming Equipment:** Digital tractors and tools will interact with the virtual crops and fields.

Let's gear up, roll up our sleeves, and dive in. 🌱🚜🌾

## Initialising simulation code

In order for the simulation to run, we have to install the `Pygame` package. This Python package is used to draw graphics for games written in Python, and we'll use it to see the results of our code.

First, we install the `Pygame` package with pip:

In [1]:
!poetry add pygame

The following packages are already present in the pyproject.toml and will be skipped:

  • pygame

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


Then run some code to set up the visuals. 

Before scrolling down, remember that we **don't have to understand every line of code in the cell below**. 

Scroll down and try to spot **variables**, **functions**, **conditional flow**, **loops**, **datatypes**, and so on.

By the end of the course, all of this code will make sense. But for now, we can just execute this cell.

In [2]:
# DO NOT CHANGE ANYTHING IN THIS CELL, ONLY EXECUTE IT!

import pygame
import logging
import random


# Constants for the tile size
TILE_SIZE = 16
UI_WIDTH = 200                  # Width of the UI section
UI_HEIGHT = 600                 # Height
TRACTOR_SIZE = 12
WHITE = (255, 255, 255)
LIGHT_BROWN = (210, 180, 140)   # Light brown for un-ploughed soil
DARK_BROWN = (101, 67, 33)      # Dark brown for ploughed soil
GREEN = (0, 255, 0)             # Green for planted crops
RED = (255, 0, 0) 
TILE_SIZE = 16                  # Constants for the tile size

#Default field
field_height = 20 # min 20
field_width = 20 # min 1


#Class declarations
class Field:
    """
    Represents a virtual field with custom dimensions and layout.

    This class creates a field represented as a two-dimensional grid. Each cell of the grid can contain different types of tiles, such as grass, soil, and various border elements. The field's dimensions are specified at the time of object creation. The class automatically generates the layout of the field, including the borders and inner soil tiles with random variations.

    Attributes:
        width (int): The width of the field specified by the user, excluding the outer grass layer.
        height (int): The height of the field specified by the user, excluding the outer grass layer.
        layout (list): A 2D list representing the field's layout, including the outer grass layer and the specified soil variations.

    Methods:
        generate_field(height, width): Generates the complete layout of the field with specified height and width. It includes an additional outer layer for grass borders and corners and fills the interior with random soil variations.

    Example:
        >>> field = Field(5, 5)
        >>> field.layout
        [['soil_bottom_right', 'soil_bottom', ...], [...], ...]
    """

    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.layout = self.generate_field(height, width)

    def generate_field(self, height, width):
        # Adjust height and width to include the outer grass layer
        height += 2
        width += 2

        # initialise the field as a 2D array filled with grass
        field = [['grass' for _ in range(width)] for _ in range(height)]

        # Define the tile names for the borders, corners, and soil variations
        grass_corner_tl = 'soil_bottom_right'
        grass_corner_tr = 'soil_bottom_left'
        grass_corner_bl = 'soil_top_right'
        grass_corner_br = 'soil_top_left'
        grass_edge_top = 'soil_bottom'
        grass_edge_bottom = 'soil_top'
        grass_edge_left = 'soil_right'
        grass_edge_right = 'soil_left'
        soil_variations = ['soil1', 'soil2', 'soil3']

        # Fill in the corners
        field[1][1] = grass_corner_tl
        field[1][width - 2] = grass_corner_tr
        field[height - 2][1] = grass_corner_bl
        field[height - 2][width - 2] = grass_corner_br

        # Fill in the borders
        for x in range(2, width - 2):
            field[1][x] = grass_edge_top
            field[height - 2][x] = grass_edge_bottom
        for y in range(2, height - 2):
            field[y][1] = grass_edge_left
            field[y][width - 2] = grass_edge_right

        # Fill in the interior with random soil variations
        for y in range(2, height - 2):
            for x in range(2, width - 2):
                field[y][x] = random.choice(soil_variations)

        return field

    def plough_field(self, row, col):
        # Check if the row and col indices are within the valid range
        if 0 <= row < len(self.layout) and 0 <= col < len(self.layout[0]):
            # Adjust for border offset
            self.layout[row + 2][col + 2] = 'ploughed_soil'
        else:
            # Optionally log an error or handle the invalid index case
            logging.error(f"Invalid plough field indices: row {row}, col {col}")

    def get_truncated_layout(self):
        return [row[2:-2] for row in self.layout[2:-2]]

    def render(self, screen, tileset):
        for y, row in enumerate(self.layout):
            for x, tile_name in enumerate(row):
                if tile_name == 'ploughed_soil':
                    screen.blit(darker_soil_tile, (x * TILE_SIZE, y * TILE_SIZE))
                else:
                    render_tile(screen, tile_name, x, y, tileset)

class Tractor:
    """
    Represents a tractor in a virtual field environment.

    This class creates a tractor object that can be drawn on a Pygame window. 
    The tractor's position is defined by its row and column in a grid layout. 
    Additionally, the tractor's appearance is determined by its colour, with a corresponding image loaded based on the specified colour.

    Attributes:
        row (int): The row position of the tractor in the grid.
        col (int): The column position of the tractor in the grid.
        border_offset (int): The offset to account for the border when drawing the tractor.
        tractor_colour (str): The colour of the tractor, used to load the correct image.
        tractor_image (Surface): The Pygame Surface object representing the tractor's image.

    Methods:
        draw(win): Draws the tractor on the given Pygame window at its current position.
        move_to(position): Updates the tractor's row and column to a new position.

    Example:
        >>> tractor = Tractor(3, 4, 'red')
        >>> tractor.move_to((5, 6))
        >>> tractor.draw(pygame_window)
    """

    def __init__(self, row, col, tractor_colour):
        self.row = row
        self.col = col
        self.border_offset = 2
        self.tractor_colour = tractor_colour
        # Load the tractor image based on the specified colour
        self.tractor_image = pygame.image.load(f'assets/{tractor_colour}_tractor_12x12.png')

    def draw(self, win):
        # Adjust position to include border offset
        x = (self.col + self.border_offset) * TILE_SIZE + (TILE_SIZE - TRACTOR_SIZE) // 2
        y = (self.row + self.border_offset) * TILE_SIZE + (TILE_SIZE - TRACTOR_SIZE) // 2
        # Draw the tractor image
        win.blit(self.tractor_image, (x, y))

    def move_to(self, position):
        self.row, self.col = position

class Button:
    """
    Represents a button in a Pygame application.

    This class creates a button that can be drawn on a Pygame window. 
    The button's position and size are defined upon creation. 
    The class also provides functionality to check whether the button has been clicked, 
    based on the mouse position.

    Attributes:
        rect (Rect): A Pygame Rect object representing the button's position and size.
        text (str): The text displayed on the button.

    Methods:
        draw(screen): Draws the button on the given Pygame screen, including the text.
        is_clicked(pos): Determines if the button is clicked, given the mouse position.

    Example:
        >>> button = Button(100, 150, 200, 50, 'Start')
        >>> button.draw(pygame_screen)
        >>> if button.is_clicked(pygame.mouse.get_pos()):
        >>>     print("Button clicked!")
    """   

    def __init__(self, x, y, width, height, text):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text

    def draw(self, screen):
        # Draw the button
        pygame.draw.rect(screen, (59, 130, 246), self.rect)
        font = pygame.font.Font(None, 30)
        text_surf = font.render(self.text, True, (255, 255, 255))
        text_rect = text_surf.get_rect(center=self.rect.center)
        screen.blit(text_surf, text_rect)

    def is_clicked(self, pos):
        return self.rect.collidepoint(pos)
    
#Function declarations

def get_tile_by_name(tileset, name, tile_width, tile_height):
    """
    Extracts a specific tile image from a larger tileset based on a descriptive name.

    This function locates the tile within the tileset using a predefined dictionary of tile names and their corresponding coordinates. It then creates and returns an image of the specified tile.

    Parameters:
        tileset (Surface): The Pygame Surface object representing the entire tileset image.
        name (str): The descriptive name of the tile to be extracted.
        tile_width (int): The width of each individual tile in the tileset.
        tile_height (int): The height of each individual tile in the tileset.

    Returns:
        Surface: A Pygame Surface object representing the extracted tile image.

    Example:
        >>> tile_image = get_tile_by_name(my_tileset, 'grass', 32, 32)
    """
    
    x, y = reversed_tile_names[name]
    rect = pygame.Rect(x * tile_width, y * tile_height, tile_width, tile_height)
    image = pygame.Surface(rect.size, pygame.SRCALPHA).convert_alpha()
    image.blit(tileset, (0, 0), rect)
    return image

def render_tile(screen, tile_name, x, y, tileset):
    """
    Renders a single tile on the screen at a specified location.

    If a tile name is provided, the function extracts the corresponding tile image from the tileset and displays it at the specified (x, y) coordinates on the screen.

    Parameters:
        screen (Surface): The Pygame Surface object representing the screen where the tile will be rendered.
        tile_name (str): The descriptive name of the tile to be rendered.
        x (int): The x-coordinate on the screen where the tile will be placed.
        y (int): The y-coordinate on the screen where the tile will be placed.
        tileset (Surface): The Pygame Surface object representing the entire tileset image.

    Example:
        >>> render_tile(game_screen, 'soil', 5, 5, my_tileset)
    """

    if tile_name:  # If there's a tile name present
        tile_image = get_tile_by_name(tileset, tile_name, TILE_SIZE, TILE_SIZE)
        screen.blit(tile_image, (x * TILE_SIZE, y * TILE_SIZE))

def start_tractor(field):
    """
    initialises a tractor and calculates its movements based on the provided field.

    The function creates a Tractor object with a starting position within the ploughable area of the field. It then determines the tractor's movement path by executing a student-defined function.

    Parameters:
        field (list): A 2D list representing the field layout.

    Returns:
        tuple: A tuple containing the Tractor object and a list of all movements it should make.

    Example:
        >>> tractor, movements = start_tractor(my_field)
    """  

    # Starting position within the ploughable area
    tractor = Tractor(2, 2, tractor_colour)  # No need to offset by border size anymore
    plough_movements = student_function(field)
    all_movements = plough_movements
    return tractor, all_movements

def render_score(screen, score, x, y):
    """
    Renders the current score on the screen at the specified location.

    This function displays the score on the screen using Pygame's font rendering system. The score is displayed at the given (x, y) coordinates.

    Parameters:
        screen (Surface): The Pygame Surface object representing the screen where the score will be rendered.
        score (int): The current score to be displayed.
        x (int): The x-coordinate on the screen where the score will be displayed.
        y (int): The y-coordinate on the screen where the score will be displayed.

    Example:
        >>> render_score(game_screen, 100, 10, 10)
    """

    font = pygame.font.Font(None, 36)
    text = font.render(f"Score: {score}", True, (0, 0, 0))
    screen.blit(text, (x, y))

def print_field(field):
    """
    Prints the current status of the field in a readable format as a list of lists inside another list.
    
    :param field: Field object
    """

    truncated_layout = field.get_truncated_layout()
    for i, row in enumerate(truncated_layout):
        row_str = ', '.join([cell.replace('soil1', 'soil').replace('soil2', 'soil').replace('soil3', 'soil') for cell in row])
        if i == 0:
            print(f"[[{row_str}],")
        elif i == len(truncated_layout) - 1:
            print(f" [{row_str}]]")
        else:
            print(f" [{row_str}],")

def print_movement(field):
    """
    This function abstracts the field and function calls
    """
    print(student_function(field.get_truncated_layout()))
    
def student_function(field):
    """
    This function is a dummy student function to init the first time.
    """
    plough_movements = []
    for row in range(len(field)):
        plough_movements.append((row, 0))
    return plough_movements

#MAIN CODE

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialise Pygame
pygame.init()

# Define the tile names and their corresponding positions in the tileset image
tile_names = {
    (0, 0): "soil_top_left", (2, 0): "soil_top_right",
    #(3, 0): "diag_tl__br", (4, 0): "diag_bl_tr", 
    (5, 0): "soil1", (6, 0): "soil2", (7, 0): "soil3",
    (0, 1): "soil_left", (1, 1): "grass", (2, 1): "soil_right", (3, 1): "soil_bottom_right", (4, 1): "soil_bottom_left", (6, 1): "soil_top_left", (7, 1): "soil_top_right", (8, 1): "soil_top", #(10, 1): "soil_left", 
    (1, 2): "soil_bottom"
    # Note that positions with empty strings are omitted
}
# Reverse the mapping to get positions from names
reversed_tile_names = {v: k for k, v in tile_names.items()}

# Load the darker soil tile
darker_soil_tile = pygame.image.load('assets/soil_tile_darker.png')

# Instantiate the field
field = Field(field_width + 2, field_height + 2)
field_layout = field.get_truncated_layout()
movement_list = student_function(field.get_truncated_layout())

#This is the main game code that is executed when we run: if __name__ == "__main__": main()
def main():
    global movement_list
    pygame.init()
    clock = pygame.time.Clock()

    # Load the tileset image
    tileset = pygame.image.load('assets/free.png')  # Replace with the actual path to your tileset image
    eai_image = pygame.image.load('assets/EAI_Blue_Dark.png')

    # Scale down the EAI image
    scale_factor = 0.3  # Example scale factor
    scaled_eai_width = int(eai_image.get_width() * scale_factor)
    scaled_eai_height = int(eai_image.get_height() * scale_factor)
    eai_image = pygame.transform.scale(eai_image, (scaled_eai_width, scaled_eai_height))

    # Initialise variables
    tractor = None
    current_movement_list = []
    movement_index = 0
    tractor_active = False
    score = 0
    score_shift = 15

    # Calculate the display size (including UI)
    rows, cols = len(field.layout) - 4, len(field.layout[0]) - 4
    display_width = TILE_SIZE * (cols + 4) + UI_WIDTH
    display_height = TILE_SIZE * (rows + 4)
    if display_height < 400:
        display_height = 400
    screen = pygame.display.set_mode((display_width, display_height))

    ui_center_x = TILE_SIZE * cols + (UI_WIDTH // 2)

    # Initialise buttons
    button_width, button_height = 100, 50
    start_button = Button(ui_center_x + 13, 100, button_width, button_height, "Start")
    stop_button = Button(ui_center_x + 13, 200, button_width, button_height, "Stop")

    # Position for the EAI image
    eai_image_x = ui_center_x + 60 - scaled_eai_width // 2
    eai_image_y = 10

    # Main game loop
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if start_button.is_clicked(event.pos):
                    print("Starting Tractor")
                    tractor, current_movement_list = start_tractor(field.get_truncated_layout())
                    movement_index = 0
                    tractor_active = True
                elif stop_button.is_clicked(event.pos):
                    print("Stop button clicked")
                    running = False

        screen.fill(WHITE)  # Clear the screen

        # Draw the EAI image at the top
        screen.blit(eai_image, (eai_image_x, eai_image_y))

        # Draw the buttons
        start_button.draw(screen)
        stop_button.draw(screen)

        # Render the field
        field.render(screen, tileset)

        # Render score
        render_score(screen, score, ui_center_x + score_shift, 300)

        # Animate tractor movement
        if tractor_active and tractor:
            if movement_index < len(current_movement_list):
                # Move the tractor
                prev_position = tractor.row, tractor.col
                next_position = current_movement_list[movement_index]
                tractor.move_to(next_position)

                # Plough the field at the tractor's new position
                field.plough_field(next_position[0], next_position[1])

                # Update score
                score += 1
                if next_position[0] != prev_position[0]:
                    if (next_position[0] % 2 == 0 and next_position[1] != 0) or \
                    (next_position[0] % 2 == 1 and next_position[1] != len(field.get_truncated_layout()[0]) - 1):
                        score += 20  # Apply penalty
                
                movement_index += 1
            else:
                tractor_active = False

        # Draw the tractor at its current position
        if tractor:
            tractor.draw(screen)

        pygame.display.flip()
        clock.tick(simulation_speed)

    pygame.quit()

pygame 2.5.2 (SDL 2.28.2, Python 3.11.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


> ⚠️ If we get any errors after running the cell above, we have to make sure we have installed `Pygame`, and extracted all of the files in the `asset` folder.

Next up are some parameters we can change. 

- If we run the simulation and the `Pygame` screen is too large, we can reduce the `field_height` and `field_width` parameters. 

- If the simulation is slowing our computers down, decrease the `simulation_speed`

- Finally, we have several options to customise our tractors. `tractor_colour` specifies which colour we want, and the options are:

 `['blue', 'green', 'orange', 'pink', 'purple', 'red']`
 
Simply change the string below to one of these options.


In [3]:
simulation_speed = 40
tractor_colour = 'pink'
field_height = 5 # min 1
field_width =  5 # min 1

## Creating a digital field

As we continue our journey in creating a digital twin for Maji Ndogo's agriculture, we now focus on a crucial aspect: programming the movement of our virtual tractor across a field. 

We can represent a field as cells:

<div align="center" style="font-size: 80%; text-align: center; margin: 0 auto">
  <figure>
    <img src="https://github.com/Explore-AI/Pictures/blob/master/alx_ds_python/Code_challenges/Field.png?raw=true"
         style="display: block; margin-left: auto; margin-right: auto; width: 25%; height: auto;"/>
    <figcaption>Figure 1: Representing a field digitally</figcaption>
  </figure>
</div>

The field above is 5x5, meaning it has a width of 5 and a height of 5 cells. To digitally represent the field, each cell can be a string `'soil'`, and all of the strings can be placed in a list to make a row in the field, `['soil', 'soil', 'soil', 'soil', 'soil']`.  If we then nest these in another list, we can represent the field as a list of lists:

In [4]:
field_5x5 = [
 ['soil', 'soil', 'soil', 'soil', 'soil'],
 ['soil', 'soil', 'soil', 'soil', 'soil'],
 ['soil', 'soil', 'soil', 'soil', 'soil'],
 ['soil', 'soil', 'soil', 'soil', 'soil'],
 ['soil', 'soil', 'soil', 'soil', 'soil'],
]

To create a field in this code challenge, we can use the command `field = Field(field_width + 2, field_height + 2)` with `field_height = 10`
`field_width =  5`, we will create a nested list of lists where each row represents the width of the field, and all of the rows combined give the height. 

If we print out the field with the command `print_field(field)` we can see that each element in the list is a string, `soil`.

In [5]:
field_height = 10
field_width =  5

field = Field(field_width + 2, field_height + 2) # We create a new field with this line that is two tiles wider than the values we gave, which makes room for some grass.
print_field(field)

[[soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil]]


We can change the field dimensions by altering the field's parameters.

In [6]:
field_height = 5
field_width =  6

field = Field(field_width + 2, field_height + 2)
print_field(field)

[[soil, soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil, soil]]


Now the field is 6x5.

## Moving through our digital field

We can control the movement of the tractor in the field using `student_function()`. This function takes a field in the format above as input and creates a list of coordinates as output where the tractor should move to called `plough_movements`.

In the example below, the function takes in a 5x5 `field`, and moves through the first row of the field (`field[0]`). `student_function()` returns a list of `plough_movements` which are tuples of the indexes of the field to where the tractor should move `(col, row)`.

In [7]:
field_height = 5
field_width =  5

field = Field(field_width + 2, field_height + 2)
print_field(field)

def student_function(field):
    # Example function - we will write our own functions
    plough_movements = []
    for row in range(len(field[0])): # Create a list of length of field[0]
        plough_movements.append((0,row))

    return plough_movements

print_movement(field)

[[soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil]]
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]


**Note:** This function loops through the indexes from `0` to `range(len(field[0]))` - the length of the row (`[0, 1, 2, 3, 4]`). For each element in the row, we append the `plough_movements` list with a tuple, `(row, col)`.

The output, `[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]`, means that the tractor starts at `(0, 0)` (always top left), then moves to position `(0, 1)`, which is to the right, until it gets to `(0, 4)`. 

<div align="center" style="font-size: 80%; text-align: center; margin: 0 auto">
  <figure>
    <img src="https://github.com/Explore-AI/Pictures/blob/master/alx_ds_python/Code_challenges/Field_plough_single.png?raw=true"
         style="display: block; margin-left: auto; margin-right: auto; width: 20%; height: auto;"/>
    <figcaption>Figure 2: Ploughing the first row</figcaption>
  </figure>
</div>




We can run the simulation by executing the cell below. A new window should pop up, and if we click on `Start`, the tractor will plough the first row. To stop the tractor, click on `Stop`.

Running the code below should open a new window, sometimes in the background, so if no window appears, look for it on your taskbar.

In [8]:
field = Field(field_width + 2, field_height + 2)

if __name__ == "__main__":
    main()

Starting Tractor
Stop button clicked


If we wanted to plough only the second row, we could use the column index, 1:

In [9]:
def student_function(field):
    plough_movements = []
    for row in range(len(field[1])):
        plough_movements.append((1,row))
    return plough_movements

print_movement(field)

[(1, 0), (1, 1), (1, 2), (1, 3), (1, 4)]


If we run simulation again:

In [10]:
field = Field(field_width + 2, field_height + 2)

if __name__ == "__main__":
    main()

Starting Tractor
Stop button clicked


We can see the tractor only ploughs the second row, index 1.

<br>

Of course we want to plough the whole field, so we can use a nested loop to do that. If we add an outer loop to the one we have, we can loop through the rows, one by one, and for each row, plough the field.

## Challenge 1: Plough the whole field

Your task is to write a function `student_function()` that takes the field as input and returns a list of coordinates in tuple format. The coordinates should represent the series of movements the tractor should make.

Rearrange the code below to make the tractor plough the entire field, row by row.

### Answer

In [12]:
def student_function(field):
    plough_movements = []
    
    for col in range(len(field)):
        for row in range(len(field[col])):
            plough_movements.append((col, row))
    return plough_movements


### Expected outputs

**Input 1:**

In [13]:
# soil1, soil2 and soil3 are used to draw variations of a soil tile in Pygame. They have no meaning to our code.

field_standard = [
    ['soil3', 'soil3', 'soil1', 'soil2', 'soil2'],  
    ['soil2', 'soil3', 'soil2', 'soil1', 'soil3'],
    ['soil2', 'soil3', 'soil1', 'soil3', 'soil3'],
    ['soil3', 'soil2', 'soil3', 'soil3', 'soil2'],
    ['soil3', 'soil2', 'soil1', 'soil3', 'soil2'],
    ['soil2', 'soil3', 'soil3', 'soil1', 'soil3'],
    ['soil3', 'soil1', 'soil3', 'soil2', 'soil2'],
    ['soil2', 'soil1', 'soil2', 'soil2', 'soil1'],
    ['soil1', 'soil1', 'soil3', 'soil2', 'soil3'],
    ['soil3', 'soil3', 'soil3', 'soil1', 'soil1']
]

student_function(field_standard)

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (6, 0),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (7, 0),
 (7, 1),
 (7, 2),
 (7, 3),
 (7, 4),
 (8, 0),
 (8, 1),
 (8, 2),
 (8, 3),
 (8, 4),
 (9, 0),
 (9, 1),
 (9, 2),
 (9, 3),
 (9, 4)]

**Output:** 

`[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4)]`

<br>


**Input 2:**

In [14]:
field_wide = [
    ['soil3', 'soil3', 'soil1', 'soil2', 'soil2', 'soil3', 'soil3', 'soil1', 'soil2', 'soil2', 'soil3', 'soil3', 'soil1', 'soil2', 'soil2'],
    ['soil3', 'soil3', 'soil1', 'soil2','soil2', 'soil3', 'soil3']
]

student_function(field_wide)

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6)]

**Output:** 

`[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6)]`

### Simulation

If our code gives the correct outputs, we can run the simulation to see if the tractor did what we expected. We should see the tractor moving from left to right, row by row until it reaches the last cell. This is what the loop is doing too. Each cell in a row is ploughed, one by one, row by row. 

In [16]:
field = Field(field_width + 2, field_height + 2)

if __name__ == "__main__":
    main()

Starting Tractor
Starting Tractor
Starting Tractor
Starting Tractor
Starting Tractor
Stop button clicked


Before contiunuing, make sure to understand how the student function was able to create the output.

## Challenge 2: Turning the tractor around

Tractors in real fields don't just move in one direction like this:

<div align="center" style="font-size: 80%; text-align: center; margin: 0 auto">
  <figure>
    <img src="https://github.com/Explore-AI/Pictures/blob/master/alx_ds_python/Code_challenges/Field_plough_standard.png?raw=true"
         style="display: block; margin-left: auto; margin-right: auto; width: 20%; height: auto;"/>
    <figcaption>Figure 3: Ploughing the whole field</figcaption>
  </figure>
</div>

They turn around at the end of each row:

<div align="center" style="font-size: 80%; text-align: center; margin: 0 auto">
  <figure>
    <img src="https://github.com/Explore-AI/Pictures/blob/master/alx_ds_python/Code_challenges/Field_plough_real.png?raw=true"
         style="display: block; margin-left: auto; margin-right: auto; width: 20%; height: auto;"/>
    <figcaption>Figure 4: Realistic tractor movements</figcaption>
  </figure>
</div>

Let’s explore how we can recreate this behaviour digitally. To make it more challenging, we introduced a penalty to the score. If we plow a row and start a new row from the left each time(as shown in Figure 3), we receive a penalty of +20 added to the score. For each row, the score will be 5 + 20, resulting in a high score. However, if we plow the field in alternating directions, there is no penalty applied, so we can aim for a lower score.

**Discussing approaches**
- **List reversal:** For odd rows, reverse the order of columns traversed.
- **If-Else condition:** We can use if-else conditions to change the direction of movement based on whether the row number is even or odd.

So let's start with creating a function that will take a list, reverse the order, and return a list. This is a very common task in programming, so search around to find out how to do this.

### Answer

In [17]:
def reverse_list(input_list):
    input_list.reverse()
    return input_list

### Expected outputs

**Input 1:**

In [18]:
current_row = [0, 1, 2, 3, 4]

reverse_list(current_row)

[4, 3, 2, 1, 0]

**Output:**

`[4, 3, 2, 1, 0]`

<br>

**Input 2:**

In [19]:
current_row = [1, 3, 2, 0, 4]

reverse_list(current_row)

[4, 0, 2, 3, 1]

**Output:**

`[4, 0, 2, 3, 1]`

## Challenge 3: Ploughing a field realistically

In this challenge, your task is to create a function that simulates the movement of a tractor across a field in the most efficient manner. The tractor should move from left to right across each row and then reverse direction for the next row. Here's how to approach it:

- **Understand the field:** Recognise that the field is a nested list of lists where each element can be accessed using row and column indexes.

- **Plan the path:** On even rows (0, 2, 4, ...), the tractor should move from left to right. On odd rows (1, 3, 5, ...), it should move from right to left.

- **Implement the Logic:** Use loops and conditions to build the path according to the above plan.

- **Reuse the `reverse_list` function:** Change the direction of the tractor by reusing code.

- **Return the movements:** Your function should ultimately return a list of tuples, with each tuple representing the coordinates (row, column) of each movement.

Remember, the goal is to translate a real-world problem into a digital format and solve it using programming.

**Hint:** You can begin by using the `tractor_movement_standard()` as a starting point for your implementation.

### Answer

In [23]:
def student_function(field):
    output = []
    for field_index, field_name in enumerate(field):
        if (0 <= field_index and field_index % 2 == 1):
            for inner_index in range(len(field_name)-1,-1,-1):
                output.append((field_index,inner_index))
        else:
            for inner_index in range(len(field_name)):
                output.append((field_index,inner_index))

    return output

### Expected outputs

**Input 1:**

In [24]:
field_standard = [
    ['soil3', 'soil3', 'soil1', 'soil2', 'soil2'],
    ['soil2', 'soil3', 'soil2', 'soil1', 'soil3'],
    ['soil2', 'soil3', 'soil1', 'soil3', 'soil3'],
    ['soil3', 'soil2', 'soil3', 'soil3', 'soil2'],
    ['soil3', 'soil2', 'soil1', 'soil3', 'soil2'],
    ['soil2', 'soil3', 'soil3', 'soil1', 'soil3'],
    ['soil3', 'soil1', 'soil3', 'soil2', 'soil2'],
    ['soil2', 'soil1', 'soil2', 'soil2', 'soil1'],
    ['soil1', 'soil1', 'soil3', 'soil2', 'soil3'],
    ['soil3', 'soil3', 'soil3', 'soil1', 'soil1']
]

student_function(field_standard)

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 4),
 (1, 3),
 (1, 2),
 (1, 1),
 (1, 0),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 4),
 (3, 3),
 (3, 2),
 (3, 1),
 (3, 0),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (5, 4),
 (5, 3),
 (5, 2),
 (5, 1),
 (5, 0),
 (6, 0),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (7, 4),
 (7, 3),
 (7, 2),
 (7, 1),
 (7, 0),
 (8, 0),
 (8, 1),
 (8, 2),
 (8, 3),
 (8, 4),
 (9, 4),
 (9, 3),
 (9, 2),
 (9, 1),
 (9, 0)]

**Output**:

`[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 4), (3, 3), (3, 2), (3, 1), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (5, 4), (5, 3), (5, 2), (5, 1), (5, 0), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (7, 4), (7, 3), (7, 2), (7, 1), (7, 0), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (9, 4), (9, 3), (9, 2), (9, 1), (9, 0)]` 

Note the tractor goes to the next row, but stays at index 4: `...(0, 4), (1, 4)...`

<br>

**Input 2:**

In [25]:
field_5x5 = [
 ['soil', 'soil', 'soil', 'soil', 'soil'],
 ['soil', 'soil', 'soil', 'soil', 'soil'],
 ['soil', 'soil', 'soil', 'soil', 'soil'],
 ['soil', 'soil', 'soil', 'soil', 'soil'],
 ['soil', 'soil', 'soil', 'soil', 'soil'],
]

student_function(field_5x5)

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 4),
 (1, 3),
 (1, 2),
 (1, 1),
 (1, 0),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 4),
 (3, 3),
 (3, 2),
 (3, 1),
 (3, 0),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4)]

**Output**:

`[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 4), (3, 3), (3, 2), (3, 1), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)]`

Finally, increase the size of the field to 20x25 and run the simulation again to see the fruits of our hard work!

In [26]:
field_height = 20
field_width =  25

field = Field(field_width + 2, field_height + 2)

if __name__ == "__main__":
    main()

Starting Tractor
Stop button clicked


Remember to transfer your answers to the `GCC_Submit_loops_and_functions.ipynb` file and submit the file.

<br>

#  

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/EAI_Blue_Dark.png"  style="width:200px";/>
</div>